In [90]:
import numpy as np
from scipy import sparse
import os
from utils import optm
from utils.functions import *
import numpy as np
from scipy import sparse
from importlib import reload



I = np.identity(2)
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


Sz = sparse.csr_matrix(Sz)
Sx = sparse.csr_matrix(Sx)
Sy = sparse.csr_matrix(Sy)
I = sparse.csr_matrix(I)

h1 = -(sparse.kron(sparse.kron(I,Sz,format='csr'), sparse.kron(Sz,I,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(I,Sx,format='csr'), sparse.kron(Sx,I,format='csr'),format='csr')
       - sparse.kron(sparse.kron(I,Sy,format='csr'), sparse.kron(Sy,I,format='csr'),format='csr') 
     ).real


h2 = -(sparse.kron(sparse.kron(Sz,I,format='csr'), sparse.kron(Sz,I,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(Sx,I,format='csr'), sparse.kron(Sx,I,format='csr'),format='csr')
       - sparse.kron(sparse.kron(Sy,I,format='csr'), sparse.kron(Sy,I,format='csr'),format='csr') 
     ).real


h3 = -(sparse.kron(sparse.kron(Sz,Sz,format='csr'), sparse.kron(I,I,format='csr'),format='csr') 
       + sparse.kron(sparse.kron(Sx,Sx,format='csr'), sparse.kron(I,I,format='csr'),format='csr')
       + sparse.kron(sparse.kron(Sy,Sy,format='csr'), sparse.kron(I,I,format='csr'),format='csr') 
     ).real

h4 = -(sparse.kron(sparse.kron(I,I,format='csr'), sparse.kron(Sz,Sz,format='csr'),format='csr') 
       + sparse.kron(sparse.kron(I,I,format='csr'), sparse.kron(Sx,Sx,format='csr'),format='csr')
       + sparse.kron(sparse.kron(I,I,format='csr'), sparse.kron(Sy,Sy,format='csr'),format='csr') 
     ).real




h1_ = -(sparse.kron(sparse.kron(Sz,I,format='csr'), sparse.kron(I,Sz,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(Sx,I,format='csr'), sparse.kron(I,Sx,format='csr'),format='csr')
       - sparse.kron(sparse.kron(Sy,I,format='csr'), sparse.kron(I,Sy,format='csr'),format='csr') 
     ).real

h2_ = -(sparse.kron(sparse.kron(I,Sz,format='csr'), sparse.kron(I,Sz,format='csr'),format='csr') 
       - sparse.kron(sparse.kron(I,Sx,format='csr'), sparse.kron(I,Sx,format='csr'),format='csr')
       - sparse.kron(sparse.kron(I,Sy,format='csr'), sparse.kron(I,Sy,format='csr'),format='csr') 
     ).real



h = h1 + h2

h_ = h1_ + h2_

on_site = h3/4 + h4/4

In [91]:
J1 = 0.3
X1 = torch.tensor((J1 * h + on_site).toarray())
X1_prime = X1 - torch.eye(X1.shape[0])*(torch.diag(X1).min())

X2 = torch.tensor((J1 * h_ + on_site).toarray())
X2_prime = X2 - torch.eye(X2.shape[0])*(torch.diag(X2).min())

In [115]:
reload(optm)
model, gl = optm.optim_matrix_symm([X2_prime], 10000, optm_method = torch.optim.SGD, seed = 10, lr = 0.01, weight_decay = 0)

target loss : 0.650

---------- iteration start ----------
iteration :    0   loss : 0.750
iteration : 1000   loss : 0.750
iteration : 2000   loss : 0.750
iteration : 3000   loss : 0.750
iteration : 4000   loss : 0.751
iteration : 5000   loss : 0.751
iteration : 6000   loss : 0.750
iteration : 7000   loss : 0.751
iteration : 8000   loss : 0.750
iteration : 9000   loss : 0.750

 -------------- results --------------
target loss      : 0.650
loss before optm : 0.750
loss after optm  : 0.751
